```
% head -n 10001 Liquor_Sales.csv | cat >Liquor_Sales_10K.csv
```

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_categorical as pdc
from sales_util import read_liquor_csv, infer_relation_cardinalities

In [3]:
df = read_liquor_csv("Liquor_Sales_10K")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Invoice/Item Number    10000 non-null  string  
 1   Date                   10000 non-null  category
 2   Store Number           10000 non-null  category
 3   Store Name             10000 non-null  category
 4   Address                10000 non-null  category
 5   City                   10000 non-null  category
 6   Zip Code               10000 non-null  category
 7   Store Location         10000 non-null  category
 8   County Number          9969 non-null   category
 9   County                 10000 non-null  category
 10  Category               9996 non-null   category
 11  Category Name          10000 non-null  category
 12  Vendor Number          10000 non-null  category
 13  Vendor Name            10000 non-null  category
 14  Item Number            10000 non-null  

In [4]:
df.iloc[0]

Invoice/Item Number                      S24127700024
Date                              2015-02-19 00:00:00
Store Number                                     3678
Store Name                            Smoke Shop, The
Address                               1918 SE 14TH ST
City                                       DES MOINES
Zip Code                                        50320
Store Location           POINT (-93.597011 41.570844)
County Number                                      77
County                                           Polk
Category                                      1031200
Category Name                          VODKA FLAVORED
Vendor Number                                     380
Vendor Name                 Phillips Beverage Company
Item Number                                     41783
Item Description         Uv Blue Raspberry Vodka Mini
Pack                                                6
Bottle Volume (ml)                                500
State Bottle Cost           

`Cities` and `Zip Code` relate many-to-many

In [5]:
pd.concat([
    df[df['Zip Code'] == '50322'][['Zip Code', 'City']],
    df[df['City'] == 'URBANDALE'][['Zip Code', 'City']]])

,Zip Code,City
165,50322,URBANDALE
223,50322,DES MOINES
340,50322,URBANDALE
412,50322,DES MOINES
423,50322,URBANDALE
...,...,...
9503,50322,URBANDALE
9705,50322,URBANDALE
9851,50322,URBANDALE
9912,50322,URBANDALE


Explore cardinality of relationships between categorial variables:

In [6]:
infer_relation_cardinalities(df)

,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml)
Date,0,28,28,28,23,25,24,18,18,19,19,19,19,35,33,7,8
Store Number,87,0,2,2,2,2,2,1,2,31,36,26,30,85,83,7,11
Store Name,87,3,0,3,2,2,2,2,2,31,36,26,30,85,83,7,11
Address,113,2,2,0,2,2,2,2,2,33,33,26,26,112,104,7,10
City,490,87,87,84,0,13,71,2,2,54,54,43,43,543,398,10,13
Zip Code,185,31,30,29,4,0,28,2,3,44,50,32,39,209,177,8,12
Store Location,482,94,95,105,67,66,0,41,49,59,74,40,58,530,402,8,12
County Number,695,196,194,202,17,26,160,0,2,60,62,51,53,816,607,11,14
County,694,196,194,201,17,26,160,1,0,60,60,51,51,815,606,11,14
Category,640,655,651,654,222,248,594,96,97,0,2,23,23,138,116,7,9


Many-to_One relationships:
- `Store Number` to `County Number`
- `County` to `County Number`
- `Vendor Name` to `Vendor Number`
- `Item Number` to `Pack`

All other pairs are "many-to-many".

Exploring suspiciously low cardinality between store variables:

In [7]:
df.groupby('Store Name')['Store Number'].nunique()[lambda x: x > 1]

Store Name
B and B West                            2
Bani's                                  2
Broadway Liquor                         2
Emmetsburg Food Pride                   2
Famous Liquors                          2
Gasland #102 / Burlington               2
Huber's Store Inc.                      2
Ida Grove Food Pride                    2
Jeff's Market / Blue Grass              2
Jeff's Market / Durant                  2
Jeff's Market / Wilton                  3
Kellogg Country Store                   2
Kum & Go #443 / Williamsburg            2
Kum & Go #97 / West Des Moines          2
Lake Liquors Wine and Spirits           2
Laurens Food Pride                      2
Liquor And Tobacco Outlet               2
Liquor and Tobacco Outlet /             2
No Frills Supermarkets / Council Blu    2
Osage Liquors                           2
Point Liquor & Tobacco                  2
Sac City Food Pride                     2
Sam's Mini Mart / Sioux City            2
Sauce                  

In [8]:
df[df['Address'] == "101 MAIN ST"].head(3)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
632,S16978900009,2014-01-22,4762,Huber's Store Inc.,101 MAIN ST,FORT ATKINSON,52144,POINT (-91.932448 43.14339),96,Winneshiek,...,64136,Disaronno Amaretto,12,750,13.86,20.79,2,41.58,1.5,0.40
2029,S14909200001,2013-10-02,4762,Huber's Store Inc.,101 MAIN ST,FORT ATKINSON,52144,POINT (-91.932448 43.14339),96,Winneshiek,...,69638,Dr. Mcgillicuddy's Cherry Schnapps,6,1750,14.66,21.99,6,131.94,10.5,2.77
2169,S27278900013,2015-08-12,4862,Pronto Market / Kingsley,101 MAIN ST,KINGSLEY,51028,POINT (-95.967322 42.58707400000001),75,Plymouth,...,57158,Chi-Chi's Strawberry Margarita W/tequila,6,1750,6.50,9.75,6,58.50,10.5,2.77


In [9]:
df.groupby('Item Description')['Item Number'].nunique()[lambda x: x > 1]

Item Description
1800 Reposado Tequila           2
1800 Silver Tequila             3
Absolut Citron (lemon Vodka)    2
Absolut Mandrin                 2
Absolut Swedish Vodka 80 Prf    5
                               ..
Uv Cake Vodka                   2
Uv Red (cherry) Vodka           3
Wild Turkey 101                 2
Windsor Canadian                2
Yukon Jack Canadian Liqueur     3
Name: Item Number, Length: 219, dtype: int64

Conclusions:
- A customer (`Store Number`) can have multiple stores (`Store Name`)
- `Address`es are not unique because same street & number exists in different `City`s
- `Store Name`s are reused by differnt customers (`Store Number`). E.g. franchises

In [10]:
df["Full Address"] = df[['Address', 'City', 'Zip Code']].agg(' | '.join, axis=1)
df["Store"] = df['Store Number'].apply(str)
df["Store"] = df[['Store', 'Store Name']].agg(' | '.join, axis=1)
df["Street & Zip"] = df[['Address', 'Zip Code']].agg(' | '.join, axis=1)


None of the above has low cardinality:

In [11]:
df.apply(pd.Series.nunique)

Invoice/Item Number      10000
Date                       840
Store Number              1252
Store Name                1226
Address                   1289
City                       382
Zip Code                   363
Store Location            1084
County Number               99
County                     113
Category                    73
Category Name               87
Vendor Number               70
Vendor Name                 89
Item Number               1574
Item Description          1175
Pack                        12
Bottle Volume (ml)          18
State Bottle Cost         1008
State Bottle Retail       1195
Bottles Sold                44
Sale (Dollars)            3098
Volume Sold (Liters)        97
Volume Sold (Gallons)       97
Full Address              1301
Store                     1254
Street & Zip              1296
dtype: int64

In [12]:
import sqlite3
con = sqlite3.connect("liquor.db")
cur = con.cursor()
#cur.execute("CREATE TABLE county(number INTEGER PRIMARY KEY, name TEXT)")

In [15]:
pd.options.display.max_columns = None
df[df['Item Number'] == '36886']

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Full Address,Store,Street & Zip
131,S10440400009,2013-02-05,3162,Nash Finch / Wholesale Food,807 GRANDVIEW,MUSCATINE,52761,POINT (-91.06411300000002 41.408437),70,Muscatine,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.30,4.96,12,59.52,9.00,2.38,807 GRANDVIEW | MUSCATINE | 52761,3162 | Nash Finch / Wholesale Food,807 GRANDVIEW | 52761
153,S12078100081,2013-05-07,2507,Hy-Vee Food Store #1 / Burlington,939 ANGULAR,BURLINGTON,52601,POINT (-91.110752 40.801889),29,Des Moines,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.30,4.96,6,29.76,4.50,1.19,939 ANGULAR | BURLINGTON | 52601,2507 | Hy-Vee Food Store #1 / Burlington,939 ANGULAR | 52601
386,S24556300015,2015-03-17,4249,Fareway Stores #048 / Clarinda,115 ESSIE DAVISON DR,CLARINDA,51632,POINT (-95.040293 40.724008),73,Page,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.31,4.97,6,29.82,4.50,1.19,115 ESSIE DAVISON DR | CLARINDA | 51632,4249 | Fareway Stores #048 / Clarinda,115 ESSIE DAVISON DR | 51632
500,S03392100057,2012-01-03,3920,Smokin' Joe's #11 Tobacco and Liquor,1606 CAMANCHE AVE,CLINTON,52732,POINT (-90.204748 41.827845),23,Clinton,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.30,4.96,2,9.92,1.50,0.40,1606 CAMANCHE AVE | CLINTON | 52732,3920 | Smokin' Joe's #11 Tobacco and Liquor,1606 CAMANCHE AVE | 52732
893,S18472900041,2014-04-16,2611,Hy-Vee Drugstore / Ottumwa,1140 NORTH JEFFERSON ST,OTTUMWA,52501,POINT (-92.401538 41.026754000000004),90,Wapello,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.31,4.97,12,59.64,9.00,2.38,1140 NORTH JEFFERSON ST | OTTUMWA | 52501,2611 | Hy-Vee Drugstore / Ottumwa,1140 NORTH JEFFERSON ST | 52501
1346,S22252100039,2014-11-07,2647,Hy-Vee #7 / Cedar Rapids,5050 EDGEWOOD RD,CEDAR RAPIDS,52411,POINT (-91.701581 42.030129),57,Linn,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.31,4.97,3,14.91,2.25,0.59,5050 EDGEWOOD RD | CEDAR RAPIDS | 52411,2647 | Hy-Vee #7 / Cedar Rapids,5050 EDGEWOOD RD | 52411
1878,S03446000020,2012-01-05,2514,Hy-Vee Drugstore / Marion,2790 7TH AVENUE,MARION,50129,POINT (-91.582229 42.033186),57,Linn,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.30,4.96,12,59.52,9.00,2.38,2790 7TH AVENUE | MARION | 50129,2514 | Hy-Vee Drugstore / Marion,2790 7TH AVENUE | 50129
1994,S08960700003,2012-11-14,4273,Smokin' Joe's #4 Tobacco and Liquor,2800 E UNIVERSITY AVE,DES MOINES,50317,POINT (-93.562106 41.600564),77,Polk,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.30,4.96,12,59.52,9.00,2.38,2800 E UNIVERSITY AVE | DES MOINES | 50317,4273 | Smokin' Joe's #4 Tobacco and Liquor,2800 E UNIVERSITY AVE | 50317
2279,S22881700003,2014-12-10,3890,Smokin' Joe's #7 Tobacco and Liquor,904 1ST AVE NW,CEDAR RAPIDS,52405,POINT (-91.683669 41.972773),57,Linn,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.31,4.97,12,59.64,9.00,2.38,904 1ST AVE NW | CEDAR RAPIDS | 52405,3890 | Smokin' Joe's #7 Tobacco and Liquor,904 1ST AVE NW | 52405
2836,S10746500028,2013-02-21,4349,Southside Food Mart,1101 ARMY POST RD,DES MOINES,50315,POINT (-93.628783 41.526511),77,Polk,1031100,100 PROOF VODKA,300,Mccormick Distilling Company,36886,Mccormick Vodka,12,750,3.30,4.96,24,119.04,18.00,4.76,1101 ARMY POST RD | DES MOINES | 50315,4349 | Southside Food Mart,1101 ARMY POST RD | 50315


In [16]:
profit = df['State Bottle Retail'] - df['State Bottle Cost']
profit.describe()

count    10000.000000
mean         5.011630
std         29.121267
min          0.000000
25%          2.750000
50%          3.960000
75%          5.770000
max       2900.000000
dtype: float64

In [19]:
profit.apply(lambda x: x >= 0).value_counts()

True    10000
Name: count, dtype: int64